In [ ]:
import pickle
from glob import glob
import numpy as np
from functools import reduce
import operator
import pandas as pd

# Load Metrics

In [ ]:
trained_mods_path = './models/trained/*'
mods = glob(trained_mods_path)
metric_paths = []
for mod in mods:
    metric_paths.append(glob(f'{mod}/metrics*'))
metric_paths = reduce(operator.concat, metric_paths)
metric_paths

In [ ]:
metrics = {}
for metric_path in metric_paths:
    mod = metric_path.rsplit('/')[-1].split('.')[0].split('_')[-1]
    with open(metric_path, 'rb') as f:
        met = pickle.load(f)
    metrics[mod] = met
metrics = pd.DataFrame(metrics).reset_index().rename(columns={'index': 'Type'})
metrics 

# Transform

In [ ]:
metrics = pd.melt(metrics, id_vars=['Type'], var_name='Model')
metrics

In [ ]:
cv_metrics = metrics[metrics['Type'] == 'cv']
cv_metrics = pd.concat([cv_metrics.drop(['value'], axis=1), cv_metrics['value'].apply(pd.Series)], axis=1)
cv_metrics = pd.melt(cv_metrics, id_vars=['Type', 'Model'], var_name='Fold')
cv_metrics = pd.concat([cv_metrics.drop(['value'], axis=1), cv_metrics['value'].apply(pd.Series)], axis=1)

test_metrics = metrics[metrics['Type'] == 'test']
test_metrics = pd.concat([test_metrics.drop(['value'], axis=1), test_metrics['value'].apply(pd.Series)], axis=1)

In [ ]:
metrics = pd.concat([cv_metrics, test_metrics]).reset_index(drop=True)
metrics

# Benchmark

In [ ]:
cv_metrics = metrics[metrics['Type'] == 'cv'].drop('Type', axis=1)
avg_cv_metrics = cv_metrics.groupby('Model').mean().reset_index()
avg_cv_metrics

In [ ]:
import plotly.express as px

In [ ]:
avg_cv_metrics_plot_data = pd.melt(avg_cv_metrics, id_vars='Model', var_name='Metric', value_name='Value')
fig = px.bar(avg_cv_metrics_plot_data, x='Model', y='Value', color='Metric', barmode='group')
fig

In [ ]:
# fig.write_html('./plots/benchmark.html')